## **Loading Required packages and Data**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
# Load the California Housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

## **Quick Look on Data**

In [4]:
# Convert X and y to Pandas DataFrames
X_df = pd.DataFrame(X, columns=data.feature_names)

# Display the loaded data
print("California Housing Data:")
print(X_df.head())

California Housing Data:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  


In [8]:
y_df = pd.DataFrame(y, columns=["target"])

print("\nCalifornia Housing Target:")
print(y_df.head())


California Housing Target:
   target
0   4.526
1   3.585
2   3.521
3   3.413
4   3.422


## **Splitting the Data into train and test sets**

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **Normalizing the data**

In [10]:
# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)

[[-0.326196    0.34849025 -0.17491646 ...  0.05137609 -1.3728112
   1.27258656]
 [-0.03584338  1.61811813 -0.40283542 ... -0.11736222 -0.87669601
   0.70916212]
 [ 0.14470145 -1.95271028  0.08821601 ... -0.03227969 -0.46014647
  -0.44760309]
 ...
 [-0.49697313  0.58654547 -0.60675918 ...  0.02030568 -0.75500738
   0.59946887]
 [ 0.96545045 -1.07984112  0.40217517 ...  0.00707608  0.90651045
  -1.18553953]
 [-0.68544764  1.85617335 -0.85144571 ... -0.08535429  0.99543676
  -1.41489815]]


## **Conversion into Tensors**

In [11]:
# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

print(X_train)

tensor([[-0.3262,  0.3485, -0.1749,  ...,  0.0514, -1.3728,  1.2726],
        [-0.0358,  1.6181, -0.4028,  ..., -0.1174, -0.8767,  0.7092],
        [ 0.1447, -1.9527,  0.0882,  ..., -0.0323, -0.4601, -0.4476],
        ...,
        [-0.4970,  0.5865, -0.6068,  ...,  0.0203, -0.7550,  0.5995],
        [ 0.9655, -1.0798,  0.4022,  ...,  0.0071,  0.9065, -1.1855],
        [-0.6854,  1.8562, -0.8514,  ..., -0.0854,  0.9954, -1.4149]])


## **Defining the Model**

In [12]:
# Define a simple regression model
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(10,1)
        self.smax= nn.Softmax()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.smax(x)
        return x

In [13]:
# Create the model
input_size = X_train.shape[1]
print(input_size)
model = RegressionModel(input_size)
print(model)

8
RegressionModel(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
  (smax): Softmax(dim=None)
)


## **Loss function & Optimizer**

In [14]:
# Define loss and optimizer
Lossf = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

## **Training**

In [15]:
# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = Lossf(outputs, y_train.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

c:\Users\Azam\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch [10/500], Loss: 2.4858
Epoch [20/500], Loss: 2.4858
Epoch [30/500], Loss: 2.4858
Epoch [40/500], Loss: 2.4858
Epoch [50/500], Loss: 2.4858
Epoch [60/500], Loss: 2.4858
Epoch [70/500], Loss: 2.4858
Epoch [80/500], Loss: 2.4858
Epoch [90/500], Loss: 2.4858
Epoch [100/500], Loss: 2.4858
Epoch [110/500], Loss: 2.4858
Epoch [120/500], Loss: 2.4858
Epoch [130/500], Loss: 2.4858
Epoch [140/500], Loss: 2.4858
Epoch [150/500], Loss: 2.4858
Epoch [160/500], Loss: 2.4858
Epoch [170/500], Loss: 2.4858
Epoch [180/500], Loss: 2.4858
Epoch [190/500], Loss: 2.4858
Epoch [200/500], Loss: 2.4858
Epoch [210/500], Loss: 2.4858
Epoch [220/500], Loss: 2.4858
Epoch [230/500], Loss: 2.4858
Epoch [240/500], Loss: 2.4858
Epoch [250/500], Loss: 2.4858
Epoch [260/500], Loss: 2.4858
Epoch [270/500], Loss: 2.4858
Epoch [280/500], Loss: 2.4858
Epoch [290/500], Loss: 2.4858
Epoch [300/500], Loss: 2.4858
Epoch [310/500], Loss: 2.4858
Epoch [320/500], Loss: 2.4858
Epoch [330/500], Loss: 2.4858
Epoch [340/500], Lo

## **Saving the Model weights**

In [17]:
# Save the trained model
torch.save(model.state_dict(), './models/california_housing_model.pth')

## **Load the Models Weights**

In [18]:
# Load the model for future use
loaded_model = RegressionModel(input_size)
loaded_model.load_state_dict(torch.load('./models/california_housing_model.pth'))

C:\Users\Azam\AppData\Local\Temp\ipykernel_7812\191450273.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('./models/california_ho

<All keys matched successfully>

In [19]:
# Evaluate the loaded model on the test set
with torch.no_grad():
    y_pred = loaded_model(X_test)
    mse = mean_squared_error(y_test.numpy(), y_pred.numpy())
    print(f'Mean Squared Error on Test Data (Loaded Model): {mse:.4f}')

Mean Squared Error on Test Data (Loaded Model): 2.4234


c:\Users\Azam\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
